In [1]:
import numpy as np
import cv2
from pathlib import Path
from PIL import Image
from skimage.color import rgb2gray
from skimage.feature import hog
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.metrics import accuracy_score
from joblib import dump, load
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import time
import os
import gc

# configs
DATA_DIR = Path("../data/OCT2017 /train")  # (kept exactly as you said it's correct)
TARGET_SIZE = 128
IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
BATCH_SIZE = 64  # tune for your RAM / disk
RANDOM_STATE = 192

CHECKPOINT_PATH = Path("checkpoints/tuning_state.pkl")
BEST_MODEL_PATH = Path("checkpoints/best_model.pkl")
REPORTS_DIR = Path("reports")
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
PDF_PATH = REPORTS_DIR / "feature_examples.pdf"

# dataloader etc
def load_and_preprocess_image(path, target_size=TARGET_SIZE):
    img = Image.open(path).convert("RGB")
    w, h = img.size
    scale = target_size / max(w, h)
    new_w, new_h = int(w * scale), int(h * scale)
    img = img.resize((new_w, new_h), Image.LANCZOS)
    canvas = Image.new("RGB", (target_size, target_size), (255, 255, 255))  # pad with white
    canvas.paste(img, ((target_size - new_w) // 2, (target_size - new_h) // 2))
    return np.asarray(canvas)

def enumerate_paths_and_labels(base_dir=DATA_DIR):
    classes = sorted([d.name for d in base_dir.iterdir() if d.is_dir()])
    label_map = {name: idx for idx, name in enumerate(classes)}
    paths, labels = [], []
    for name in classes:
        for p in sorted((base_dir / name).glob("*")):
            if p.suffix.lower() in IMG_EXTS:
                paths.append(p)
                labels.append(label_map[name])
    y = np.array(labels, dtype=np.int64)
    return paths, y, label_map

def batch_iterator(paths, indices=None, batch_size=BATCH_SIZE):
    if indices is None:
        indices = np.arange(len(paths))
    n = len(indices)
    for i in range(0, n, batch_size):
        idxs = indices[i:i+batch_size]
        imgs = [load_and_preprocess_image(paths[j]) for j in idxs]
        yield np.stack(imgs), idxs

# HOG
class HOGTransformer(BaseEstimator, TransformerMixin):
    """
    Converts (N, H, W, 3) uint8 images -> (N, F) HOG feature vectors (float64).
    """
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2-Hys", visualize=False):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
        self.visualize = visualize  # only used for making the PDF examples

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        feats = []
        for img in X:
            # normalize to float to avoid warnings & match expectations
            g = rgb2gray(img.astype(np.float32) / 255.0)
            f = hog(
                g,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block,
                block_norm=self.block_norm,
                feature_vector=True
            )
            feats.append(f)
        return np.vstack(feats)

    def transform_with_viz(self, img_single):
        """Return (features, hog_image) for one image, to visualize in the PDF."""
        g = rgb2gray(img_single.astype(np.float32) / 255.0)
        f, hog_img = hog(
            g,
            orientations=self.orientations,
            pixels_per_cell=self.pixels_per_cell,
            cells_per_block=self.cells_per_block,
            block_norm=self.block_norm,
            visualize=True,
            feature_vector=True
        )
        return f, hog_img

# SIFT
class SIFTTransformer(BaseEstimator, TransformerMixin):
    """
    Extracts SIFT descriptors per image and pools to a fixed-length vector:
        feature = concat(mean(desc, axis=0), std(desc, axis=0), [count])
    Resulting dimension = 128 + 128 + 1 = 257
    """
    def __init__(self, n_features=0, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6):
        # n_features=0 lets SIFT choose; you can clamp e.g., 500 to speed up
        self.n_features = n_features
        self.contrastThreshold = contrastThreshold
        self.edgeThreshold = edgeThreshold
        self.sigma = sigma
        self._sift = None

    def _ensure_sift(self):
        if self._sift is None:
            if not hasattr(cv2, "SIFT_create"):
                raise RuntimeError(
                    "SIFT not available in your OpenCV build. Install `opencv-contrib-python`."
                )
            self._sift = cv2.SIFT_create(
                nfeatures=self.n_features,
                contrastThreshold=self.contrastThreshold,
                edgeThreshold=self.edgeThreshold,
                sigma=self.sigma
            )

    def fit(self, X, y=None):
        self._ensure_sift()
        return self

    def _img_to_gray(self, img):
        if img.ndim == 3 and img.shape[2] == 3:
            return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        return img

    def transform(self, X):
        self._ensure_sift()
        out = []
        for img in X:
            g = self._img_to_gray(img)
            kps, desc = self._sift.detectAndCompute(g, None)
            if desc is None or len(desc) == 0:
                # No keypoints: zeros
                mean = np.zeros(128, dtype=np.float32)
                std = np.zeros(128, dtype=np.float32)
                count = 0.0
            else:
                mean = desc.mean(axis=0)
                std = desc.std(axis=0)
                count = float(len(desc))
            vec = np.concatenate([mean, std, np.array([count], dtype=np.float32)], axis=0)
            out.append(vec.astype(np.float32))
        return np.vstack(out)

    def detect_keypoints(self, img_single):
        """Return keypoints for visualization purposes."""
        self._ensure_sift()
        g = self._img_to_gray(img_single)
        kps = self._sift.detect(g, None)
        return kps

# HOG+SIFT
class HOGSIFTConcatTransformer(BaseEstimator, TransformerMixin):
    """
    Computes HOG and SIFT features, then concatenates them:
      X -> [HOG(X) | SIFT(X)]
    Optional per-branch weights let you emphasize one feature family.
    """
    def __init__(
        self,
        hog_params=None,
        sift_params=None,
        hog_weight=1.0,
        sift_weight=1.0,
    ):
        self.hog_params = hog_params or {}
        self.sift_params = sift_params or {}
        self.hog_weight = float(hog_weight)
        self.sift_weight = float(sift_weight)
        self._hog = HOGTransformer(**self.hog_params)
        self._sift = SIFTTransformer(**self.sift_params)

    def fit(self, X, y=None):
        # Ensure sub-transformers are ready (SIFT alloc)
        self._hog.fit(X, y)
        self._sift.fit(X, y)
        return self

    def transform(self, X):
        H = self._hog.transform(X).astype(np.float32, copy=False)
        S = self._sift.transform(X).astype(np.float32, copy=False)
        if self.hog_weight != 1.0:
            H = H * self.hog_weight
        if self.sift_weight != 1.0:
            S = S * self.sift_weight
        return np.hstack([H, S]).astype(np.float32, copy=False)

def make_pipeline(
    feat_type="hog",
    hog_params=None,
    sift_params=None,
    pca_n=None,
    C=1.0,
    hog_weight=1.0,
    sift_weight=1.0,
):
    if feat_type == "hog":
        feat = HOGTransformer(**(hog_params or {}))
    elif feat_type == "sift":
        feat = SIFTTransformer(**(sift_params or {}))
    elif feat_type in ("hog+sift", "concat", "hog_sift"):
        feat = HOGSIFTConcatTransformer(
            hog_params=(hog_params or {}),
            sift_params=(sift_params or {}),
            hog_weight=hog_weight,
            sift_weight=sift_weight,
        )
    else:
        raise ValueError("feat_type must be 'hog', 'sift', or 'hog+sift'")

    steps = [("feat", feat), ("scaler", StandardScaler(with_mean=True))]
    if pca_n is not None:
        steps.append(("pca", PCA(n_components=pca_n, svd_solver="auto", random_state=RANDOM_STATE)))
    steps.append(("svm", LinearSVC(C=C, dual=False, max_iter=10000, random_state=RANDOM_STATE)))
    return Pipeline(steps)

def _feature_dim_of_transformer(transformer, example_img):
    """Probe feature dimensionality with one image."""
    f = transformer.transform(np.expand_dims(example_img, 0))
    return f.shape[1]

def _build_feats_memmap(n_rows, n_cols, dtype=np.float32):
    tmp_dir = Path("feature_cache")
    tmp_dir.mkdir(exist_ok=True)
    path = tmp_dir / f"feats_{n_rows}x{n_cols}_{int(time.time())}.dat"
    arr = np.memmap(path, mode="w+", dtype=dtype, shape=(n_rows, n_cols))
    return arr, path

def _extract_features_batched(paths, indices, transformer, probe_img, batch_size=BATCH_SIZE, pbar_desc=None,
                              pbar_position=2, pbar_leave=False):
    fdim = _feature_dim_of_transformer(transformer, probe_img)
    feats, path = _build_feats_memmap(len(indices), fdim, dtype=np.float32)
    write_ptr = 0
    desc = pbar_desc or f"Extract {type(transformer).__name__} (n={len(indices)})"
    with tqdm(total=len(indices), desc=desc, unit="img", position=pbar_position, leave=pbar_leave, dynamic_ncols=True) as pbar:
        for Xb, idxs in batch_iterator(paths, indices, batch_size):
            Fb = transformer.transform(Xb).astype(np.float32, copy=False)
            n = len(idxs)
            feats[write_ptr:write_ptr+n] = Fb
            write_ptr += n
            pbar.update(n)
    feats.flush()
    return feats, path

def fit_eval_pipeline(paths, y, params, cv_splits=5, random_state=RANDOM_STATE):
    """
    Train/validate one param set using stratified K-fold; returns mean accuracy and per-fold scores.
    Featurization happens in batches directly from disk paths.
    """
    feat_type = params["feat__type"]  # "hog" or "sift" or "hog+sift"

    if feat_type == "hog":
        feat_params = dict(
            orientations=params["hog__orientations"],
            pixels_per_cell=params["hog__pixels_per_cell"],
            cells_per_block=params["hog__cells_per_block"],
            block_norm=params.get("hog__block_norm", "L2-Hys"),
        )
        transformer = HOGTransformer(**feat_params)

    elif feat_type == "sift":
        feat_params = dict(
            n_features=params.get("sift__n_features", 0),
            contrastThreshold=params.get("sift__contrastThreshold", 0.04),
            edgeThreshold=params.get("sift__edgeThreshold", 10),
            sigma=params.get("sift__sigma", 1.6),
        )
        transformer = SIFTTransformer(**feat_params)

    elif feat_type in ("hog+sift", "concat", "hog_sift"):
        hogp = dict(
            orientations=params["hog__orientations"],
            pixels_per_cell=params["hog__pixels_per_cell"],
            cells_per_block=params["hog__cells_per_block"],
            block_norm=params.get("hog__block_norm", "L2-Hys"),
        )
        siftp = dict(
            n_features=params.get("sift__n_features", 0),
            contrastThreshold=params.get("sift__contrastThreshold", 0.04),
            edgeThreshold=params.get("sift__edgeThreshold", 10),
            sigma=params.get("sift__sigma", 1.6),
        )
        transformer = HOGSIFTConcatTransformer(
            hog_params=hogp,
            sift_params=siftp,
            hog_weight=params.get("concat__hog_weight", 1.0),
            sift_weight=params.get("concat__sift_weight", 1.0),
        )

    else:
        raise ValueError("Unknown feat__type")

    pca_n = params.get("pca__n_components", None)
    C = params["svm__C"]

    # For probing feature dim, grab one image
    probe_img = load_and_preprocess_image(paths[0])

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
    fold_scores = []

    with tqdm(total=cv_splits, desc="CV folds", position=1, leave=False, dynamic_ncols=True) as foldbar:
        for fold_i, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y)), y), start=1):
            transformer.fit(None)  # initialize

            Xtr_feats, tr_path = _extract_features_batched(
                paths, train_idx, transformer, probe_img,
                pbar_desc=f"Extract {type(transformer).__name__} [train f{fold_i}]", pbar_position=2,
                pbar_leave=False
            )
            Xval_feats, val_path = _extract_features_batched(
                paths, val_idx, transformer, probe_img,
                pbar_desc=f"Extract {type(transformer).__name__} [val   f{fold_i}]",
                pbar_position=3,
                pbar_leave=False
            )

            # Convert to ndarray views
            Xtr = np.asarray(Xtr_feats)
            Xval = np.asarray(Xval_feats)

            # Decide SVM dual based on dimensionality
            n_tr, d_tr = Xtr.shape
            dual = d_tr > n_tr

            steps = [("scaler", StandardScaler(with_mean=True))]
            # Clamp PCA to valid range
            use_pca_n = None
            if pca_n is not None:
                max_pca = min(n_tr, d_tr) - 1
                if max_pca >= 1 and pca_n <= max_pca:
                    use_pca_n = pca_n
                else:
                    tqdm.write(f"  -> Skipping PCA (n_components={pca_n} > max={max_pca})")
            if use_pca_n is not None:
                steps.append(("pca", PCA(n_components=use_pca_n, svd_solver="auto", random_state=random_state)))
            steps.append(("svm", LinearSVC(C=C, dual=dual, max_iter=10000, random_state=random_state)))
            clf = Pipeline(steps)

            clf.fit(Xtr, y[train_idx])
            preds = clf.predict(Xval)
            acc = accuracy_score(y[val_idx], preds)
            fold_scores.append(acc)

            # Clean up memmaps safely
            try:
                Xtr_feats.flush(); del Xtr_feats
                Xval_feats.flush(); del Xval_feats
                del Xtr, Xval
                gc.collect()
                os.remove(tr_path)
                os.remove(val_path)
            except Exception:
                pass

            foldbar.update(1)

    return float(np.mean(fold_scores)), fold_scores

def save_state(state_path, state):
    tmp = state_path.with_suffix(".tmp")
    dump(state, tmp)
    os.replace(tmp, state_path)

def load_state(state_path):
    return load(state_path)

def run_tuning(
    paths, y,
    param_grid,
    checkpoint_path=CHECKPOINT_PATH,
    best_model_path=BEST_MODEL_PATH,
    cv_splits=5,
    save_every=1,
    resume=True,
    refit_best_on_full=True
):
    checkpoint_path.parent.mkdir(parents=True, exist_ok=True)
    best_model_path.parent.mkdir(parents=True, exist_ok=True)

    param_list = list(ParameterGrid(param_grid))
    print(f"\n[INFO] Generated {len(param_list)} parameter combinations to evaluate")

    if resume and checkpoint_path.exists():
        state = load_state(checkpoint_path)
        print(f"[Resume] Loaded state with {state['next_idx']} / {len(param_list)} evaluated.")
    else:
        state = dict(
            param_list=param_list,
            results=[],
            best_score=-np.inf,
            best_params=None,
            next_idx=0,
            cv_splits=cv_splits,
            started_at=time.time(),
        )
        print(f"[INFO] Starting fresh tuning run")

    last_save = 0
    total = len(param_list)

    print(f"[INFO] Will evaluate from index {state['next_idx']} to {total}")

    with tqdm(total=total, initial=state["next_idx"], desc="Tuning grid", position=0, leave=True, dynamic_ncols=True) as tbar:
        while state["next_idx"] < len(state["param_list"]):
            idx = state["next_idx"]
            params = state["param_list"][idx]
            t0 = time.time()
            tqdm.write(f"\n[{idx+1}/{len(param_list)}] Evaluating: {params}")

            try:
                mean_acc, fold_scores = fit_eval_pipeline(paths, y, params, cv_splits=cv_splits)
                dt = time.time() - t0

                state["results"].append({
                    "idx": idx,
                    "params": params,
                    "mean_accuracy": mean_acc,
                    "fold_scores": fold_scores,
                    "seconds": dt,
                })

                tqdm.write(f"  -> Completed in {dt:.1f}s, mean accuracy: {mean_acc:.4f}")
            except Exception as e:
                tqdm.write(f"  -> ERROR: {str(e)}")
                import traceback
                tqdm.write(traceback.format_exc())
                # Save error in results
                state["results"].append({
                    "idx": idx,
                    "params": params,
                    "error": str(e),
                    "seconds": time.time() - t0,
                })
                # Continue to next iteration
                state["next_idx"] += 1
                last_save += 1
                tbar.update(1)
                if last_save >= save_every:
                    save_state(checkpoint_path, state)
                    tqdm.write(f"  [Checkpoint saved after error]: {checkpoint_path}")
                    last_save = 0
                continue

            # Update progress immediately after evaluation completes
            state["next_idx"] += 1
            last_save += 1
            tbar.update(1)

            # Track best
            if mean_acc > state["best_score"]:
                state["best_score"] = mean_acc
                state["best_params"] = params
                tqdm.write(f"  -> New best: {mean_acc:.4f}")

                if refit_best_on_full:
                    tqdm.write("  -> Refit best on full data (batched features)...")
                    # Build full-feature memmap
                    feat_type = params["feat__type"]

                    if feat_type == "hog":
                        feat_params = dict(
                            orientations=params["hog__orientations"],
                            pixels_per_cell=params["hog__pixels_per_cell"],
                            cells_per_block=params["hog__cells_per_block"],
                            block_norm=params.get("hog__block_norm", "L2-Hys"),
                        )
                        transformer = HOGTransformer(**feat_params)
                    elif feat_type == "sift":
                        feat_params = dict(
                            n_features=params.get("sift__n_features", 0),
                            contrastThreshold=params.get("sift__contrastThreshold", 0.04),
                            edgeThreshold=params.get("sift__edgeThreshold", 10),
                            sigma=params.get("sift__sigma", 1.6),
                        )
                        transformer = SIFTTransformer(**feat_params)
                    elif feat_type in ("hog+sift", "concat", "hog_sift"):
                        hogp = dict(
                            orientations=params["hog__orientations"],
                            pixels_per_cell=params["hog__pixels_per_cell"],
                            cells_per_block=params["hog__cells_per_block"],
                            block_norm=params.get("hog__block_norm", "L2-Hys"),
                        )
                        siftp = dict(
                            n_features=params.get("sift__n_features", 0),
                            contrastThreshold=params.get("sift__contrastThreshold", 0.04),
                            edgeThreshold=params.get("sift__edgeThreshold", 10),
                            sigma=params.get("sift__sigma", 1.6),
                        )
                        feat_params = {
                            "hog_params": hogp,
                            "sift_params": siftp,
                            "hog_weight": params.get("concat__hog_weight", 1.0),
                            "sift_weight": params.get("concat__sift_weight", 1.0),
                        }
                        transformer = HOGSIFTConcatTransformer(**feat_params)
                    else:
                        raise ValueError(f"Unknown feat_type: {feat_type}")

                    probe_img = load_and_preprocess_image(paths[0])
                    transformer.fit(None)
                    all_indices = np.arange(len(paths))
                    X_feats, feats_path = _extract_features_batched(
                        paths, all_indices, transformer, probe_img,
                        pbar_desc="Extract features [full dataset]",
                        pbar_position=1, pbar_leave=False
                    )

                    X_all = np.asarray(X_feats)
                    n_all, d_all = X_all.shape
                    dual = d_all > n_all  # choose dual for LinearSVC

                    # Final pipeline from features onward
                    steps = [("scaler", StandardScaler(with_mean=True))]
                    pca_n = params.get("pca__n_components", None)
                    use_pca_n = None
                    if pca_n is not None:
                        max_pca = min(n_all, d_all) - 1
                        if max_pca >= 1 and pca_n <= max_pca:
                            use_pca_n = pca_n
                        else:
                            tqdm.write(f"  -> Skipping PCA (n_components={pca_n} > max={max_pca})")
                    if use_pca_n is not None:
                        steps.append(("pca", PCA(n_components=use_pca_n, svd_solver="auto", random_state=RANDOM_STATE)))
                    steps.append(("svm", LinearSVC(C=params["svm__C"], dual=dual, max_iter=10000, random_state=RANDOM_STATE)))
                    best_pipe = Pipeline(steps)
                    best_pipe.fit(X_all, y)

                    dump({
                        "feature_type": feat_type,
                        "feature_params": feat_params,
                        "post_feat_pipeline": best_pipe
                    }, best_model_path)

                    tqdm.write(f"  -> Saved best model to: {best_model_path}")

                    # Cleanup full-dataset memmap
                    try:
                        X_feats.flush(); del X_feats
                        del X_all
                        gc.collect()
                        os.remove(feats_path)
                    except Exception:
                        pass

            if last_save >= save_every:
                save_state(checkpoint_path, state)
                tqdm.write(f"  [Checkpoint saved]: {checkpoint_path}")
                last_save = 0

    save_state(checkpoint_path, state)
    print("\n==== Tuning complete ====")
    print(f"Best CV acc: {state['best_score']:.4f}")
    print(f"Best params: {state['best_params']}")
    print(f"State saved at: {checkpoint_path}")
    if refit_best_on_full and best_model_path.exists():
        print(f"Best model at: {best_model_path}")
    return state

def make_feature_examples_pdfs(paths, label_map, base_pdf_path=PDF_PATH, hog_params=None, sift_params=None):
    base = Path(base_pdf_path)
    classes = sorted({p.parent.name for p in paths})

    first_idx_by_class = {}
    for i, p in enumerate(paths):
        cls = p.parent.name
        if cls not in first_idx_by_class:
            first_idx_by_class[cls] = i
        if len(first_idx_by_class) == len(classes):
            break

    hog_t = HOGTransformer(**(hog_params or {}))
    sift_t = SIFTTransformer(**(sift_params or {}))

    for cls in classes:
        idx = first_idx_by_class.get(cls)
        if idx is None:
            print(f"[Warning] No example found for class: {cls}")
            continue

        img = load_and_preprocess_image(paths[idx])
        _, hog_img = hog_t.transform_with_viz(img)
        kps = sift_t.detect_keypoints(img)
        img_kp_rgb = cv2.cvtColor(
            cv2.drawKeypoints(cv2.cvtColor(img, cv2.COLOR_RGB2BGR), kps, None,
                              flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS),
            cv2.COLOR_BGR2RGB
        )

        cls_pdf = base.with_name(f"{base.stem}_{cls}{base.suffix or '.pdf'}")
        with PdfPages(cls_pdf) as pdf:
            fig = plt.figure(figsize=(12, 4), constrained_layout=True)
            fig.suptitle(f"Class: {cls}", fontsize=14)
            for i_ax, (title, im) in enumerate([("Padded original", img),
                                                ("HOG visualization", hog_img),
                                                (f"SIFT keypoints (n={len(kps)})", img_kp_rgb)], 1):
                ax = fig.add_subplot(1, 3, i_ax)
                ax.imshow(im)
                ax.set_title(title)
                ax.axis("off")
            pdf.savefig(fig)
            plt.close(fig)
        print(f"[PDF] Saved {cls_pdf}")


In [2]:
# Smoke test

if __name__ == "__main__":
    # ---------- QUICK SVM SMOKE TEST ----------
    # Purpose: verify that features -> LinearSVC training/eval works end-to-end
    # Uses a tiny stratified subset for speed & low memory.

    from collections import defaultdict
    from sklearn.model_selection import train_test_split

    np.random.seed(RANDOM_STATE)

    print(f"[SMOKE] Looking for data in: {DATA_DIR}")
    if not DATA_DIR.exists():
        print(f"[SMOKE][ERROR] Data directory does not exist: {DATA_DIR}")
        print(f"[SMOKE][ERROR] CWD: {Path.cwd()}")
        exit(1)

    paths, y, label_map = enumerate_paths_and_labels(DATA_DIR)
    print(f"[SMOKE] Found {len(paths)} images across {len(label_map)} classes: {label_map}")
    if len(paths) == 0:
        print("[SMOKE][ERROR] No images found! Check DATA_DIR and IMG_EXTS.")
        exit(1)

    # ---- pick a tiny, stratified subset (up to N_PER_CLASS per class) ----
    N_PER_CLASS = 15  # make smaller/larger if you like
    by_class = defaultdict(list)
    for idx, p in enumerate(paths):
        by_class[y[idx]].append(idx)

    subset_indices = []
    for cls, idxs in by_class.items():
        take = min(N_PER_CLASS, len(idxs))
        subset_indices.extend(idxs[:take])
    subset_indices = np.array(sorted(subset_indices))
    print(f"[SMOKE] Using {len(subset_indices)} images total (~{N_PER_CLASS} per class max)")

    # ---- load subset images, extract lightweight HOG features in-memory ----
    hog_params = dict(orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2-Hys")
    hog_t = HOGTransformer(**hog_params).fit(None)

    # Load images -> features
    X_imgs = []
    y_sub = y[subset_indices]
    for idx in subset_indices:
        X_imgs.append(load_and_preprocess_image(paths[idx]))
    X_imgs = np.stack(X_imgs, axis=0)
    X_feats = hog_t.transform(X_imgs).astype(np.float32)

    print(f"[SMOKE] Feature matrix: {X_feats.shape} (n_samples x n_features)")

    # ---- train/val split (stratified) ----
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_feats, y_sub, test_size=0.25, random_state=RANDOM_STATE, stratify=y_sub
    )
    print(f"[SMOKE] Train: {X_tr.shape[0]}  Val: {X_val.shape[0]}")

    # ---- minimal pipeline: scale -> (optional PCA) -> LinearSVC ----
    steps = [("scaler", StandardScaler(with_mean=True))]

    # Optional PCA: keep it small/safe (skip if invalid)
    PCA_N = 64
    max_pca = min(X_tr.shape[0], X_tr.shape[1]) - 1
    if max_pca >= 2 and PCA_N <= max_pca:
        steps.append(("pca", PCA(n_components=PCA_N, random_state=RANDOM_STATE)))
        print(f"[SMOKE] Using PCA(n_components={PCA_N})")
    else:
        print(f"[SMOKE] Skipping PCA (requested {PCA_N}, max valid {max_pca})")

    # Decide dual based on dimensionality AFTER the (optional) PCA
    # We’ll compute it from the raw features now, but if PCA is present it only helps.
    d_eff = PCA_N if ("pca",) and len(steps) > 1 and steps[-1][0] == "pca" else X_tr.shape[1]
    dual = d_eff > X_tr.shape[0]

    steps.append(("svm", LinearSVC(C=1.0, dual=dual, max_iter=10000, random_state=RANDOM_STATE)))
    clf = Pipeline(steps)

    print(f"[SMOKE] Training LinearSVC (dual={dual})...")
    clf.fit(X_tr, y_tr)

    acc = clf.score(X_val, y_val)
    print(f"[SMOKE] Validation accuracy: {acc:.4f}")

    # quick prediction demo on a couple of samples
    preds = clf.predict(X_val[:5])
    print(f"[SMOKE] First 5 preds: {preds.tolist()}")
    print(f"[SMOKE] First 5 true : {y_val[:5].tolist()}")

    print("[SMOKE] Done.")


[SMOKE] Looking for data in: ../data/OCT2017 /train
[SMOKE] Found 83484 images across 4 classes: {'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}
[SMOKE] Using 60 images total (~15 per class max)
[SMOKE] Feature matrix: (60, 8100) (n_samples x n_features)
[SMOKE] Train: 45  Val: 15
[SMOKE] Skipping PCA (requested 64, max valid 44)
[SMOKE] Training LinearSVC (dual=True)...
[SMOKE] Validation accuracy: 0.7333
[SMOKE] First 5 preds: [2, 0, 3, 0, 0]
[SMOKE] First 5 true : [2, 0, 1, 0, 0]
[SMOKE] Done.


In [2]:
# One model test

import time
import os
import gc
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from joblib import dump

# --- You already have these in your file ---
# - DATA_DIR, RANDOM_STATE, BATCH_SIZE
# - load_and_preprocess_image
# - HOGTransformer
# - _extract_features_batched

def run_full_hog_eval(
    data_dir: Path = DATA_DIR,
    test_size: float = 0.2,
    random_state: int = RANDOM_STATE,
    hog_params: dict = None,
    use_pca: bool = False,
    pca_n: int = 256,
    C: float = 1.0,
    reports_dir: Path = REPORTS_DIR,
):
    hog_params = hog_params or dict(
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys",
    )

    reports_dir.mkdir(parents=True, exist_ok=True)
    log_path = reports_dir / "full_hog_eval.txt"

    t0_total = time.time()

    print(f"[FULL] Data dir: {data_dir}")
    if not data_dir.exists():
        raise FileNotFoundError(f"Data directory does not exist: {data_dir}")

    # Enumerate dataset
    from collections import Counter
    paths, y, label_map = enumerate_paths_and_labels(data_dir)
    counts = Counter(y.tolist())
    print(f"[FULL] Found {len(paths)} images across {len(label_map)} classes: {label_map}")
    print(f"[FULL] Class counts: {dict(counts)}")

    # Stratified split
    idx_all = np.arange(len(paths))
    tr_idx, val_idx = train_test_split(
        idx_all,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )
    print(f"[FULL] Train: {len(tr_idx)}  Val: {len(val_idx)}")

    # Build transformer and probe feature dim
    transformer = HOGTransformer(**hog_params)
    probe_img = load_and_preprocess_image(paths[0])
    transformer.fit(None)

    # Extract features (batched to memmaps), with timings
    print("[FULL] Extracting HOG features (train)...")
    t1 = time.time()
    Xtr_feats, tr_path = _extract_features_batched(
        paths, tr_idx, transformer, probe_img,
        batch_size=BATCH_SIZE,
        pbar_desc="HOG [train]",
        pbar_position=2,
        pbar_leave=False,
    )
    t2 = time.time()
    print(f"[FULL] Train features shape: {tuple(np.asarray(Xtr_feats).shape)}  (took {(t2 - t1):.1f}s)")

    print("[FULL] Extracting HOG features (val)...")
    t3 = time.time()
    Xval_feats, val_path = _extract_features_batched(
        paths, val_idx, transformer, probe_img,
        batch_size=BATCH_SIZE,
        pbar_desc="HOG [val]",
        pbar_position=3,
        pbar_leave=False,
    )
    t4 = time.time()
    print(f"[FULL] Val features shape: {tuple(np.asarray(Xval_feats).shape)}  (took {(t4 - t3):.1f}s)")

    # Prepare arrays (views on memmap)
    Xtr = np.asarray(Xtr_feats)
    Xval = np.asarray(Xval_feats)
    ytr = y[tr_idx]
    yval = y[val_idx]

    # Decide dual after (optional) PCA dimension
    n_tr, d_tr = Xtr.shape
    use_pca_n = None
    if use_pca:
        max_pca = min(n_tr, d_tr) - 1
        if max_pca >= 2 and pca_n <= max_pca:
            use_pca_n = pca_n
            print(f"[FULL] Will use PCA(n_components={use_pca_n})")
        else:
            print(f"[FULL] Skipping PCA (requested {pca_n}, max valid {max_pca})")

    # Pipeline: scale -> optional PCA -> LinearSVC
    steps = [("scaler", StandardScaler(with_mean=True))]
    if use_pca_n is not None:
        steps.append(("pca", PCA(n_components=use_pca_n, random_state=random_state)))

    # Effective feature dimension for dual decision
    d_eff = use_pca_n if use_pca_n is not None else d_tr
    dual = d_eff > n_tr
    steps.append(("svm", LinearSVC(C=C, dual=dual, max_iter=10000, random_state=random_state)))
    clf = Pipeline(steps)

    print(f"[FULL] Training LinearSVC (C={C}, dual={dual}) ...")
    t5 = time.time()
    clf.fit(Xtr, ytr)
    t6 = time.time()
    print(f"[FULL] Training time: {(t6 - t5):.1f}s")

    print("[FULL] Inference on validation set ...")
    t7 = time.time()
    preds = clf.predict(Xval)
    acc = accuracy_score(yval, preds)
    t8 = time.time()
    print(f"[FULL] Validation accuracy: {acc:.4f}  (inference {(t8 - t7):.1f}s)")

    total_time = time.time() - t0_total
    print(f"[FULL] TOTAL wall-clock time: {total_time:.1f}s")

    # Save a lightweight model snapshot (post-feature pipeline) and log
    model_path = reports_dir / "full_hog_linear_svc.joblib"
    dump({"hog_params": hog_params, "post_feat_pipeline": clf}, model_path)
    print(f"[FULL] Saved model to: {model_path}")

    # Clean up memmaps
    try:
        Xtr_feats.flush(); del Xtr_feats
        Xval_feats.flush(); del Xval_feats
        del Xtr, Xval, ytr, yval
        gc.collect()
        os.remove(tr_path)
        os.remove(val_path)
    except Exception:
        pass

    # Write a small report
    with open(log_path, "w") as f:
        f.write("=== Full HOG LinearSVC Evaluation ===\n")
        f.write(f"Data dir: {data_dir}\n")
        f.write(f"Train/Val sizes: {len(tr_idx)}/{len(val_idx)} (test_size={test_size})\n")
        f.write(f"HOG params: {hog_params}\n")
        f.write(f"PCA: {'yes' if use_pca_n is not None else 'no'}  n={use_pca_n}\n")
        f.write(f"SVM: LinearSVC(C={C}, dual={dual})\n")
        f.write(f"Feature extraction (train): {(t2 - t1):.2f}s\n")
        f.write(f"Feature extraction (val):   {(t4 - t3):.2f}s\n")
        f.write(f"Training time:              {(t6 - t5):.2f}s\n")
        f.write(f"Inference time (val):       {(t8 - t7):.2f}s\n")
        f.write(f"Validation accuracy:        {acc:.4f}\n")
        f.write(f"TOTAL wall-clock:           {total_time:.2f}s\n")
        f.write(f"Model path:                 {model_path}\n")
    print(f"[FULL] Wrote summary: {log_path}")

    return {
        "train_time_s": t6 - t5,
        "val_infer_time_s": t8 - t7,
        "feat_time_train_s": t2 - t1,
        "feat_time_val_s": t4 - t3,
        "total_time_s": total_time,
        "val_accuracy": float(acc),
        "model_path": str(model_path),
        "log_path": str(log_path),
        "train_shape": (len(tr_idx), d_tr),
        "val_shape": (len(val_idx), d_tr),
    }


if __name__ == "__main__":
    # ------------ FULL HOG EVAL ENTRYPOINT ------------
    # Tweak these flags quickly from CLI by editing defaults above if desired.
    results = run_full_hog_eval(
        data_dir=DATA_DIR,
        test_size=0.2,
        random_state=RANDOM_STATE,
        hog_params=dict(
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            block_norm="L2-Hys",
        ),
        use_pca=True,     # set True to try PCA on the full run
        pca_n=256,         # only used if use_pca=True and valid (<= min(n,d)-1)
        C=1.0,             # try {0.25, 1.0, 4.0} for quick sensitivity
        reports_dir=REPORTS_DIR,
    )
    print("[FULL] Summary:", results)

[FULL] Data dir: ../data/OCT2017 /train
[FULL] Found 83484 images across 4 classes: {'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}
[FULL] Class counts: {0: 37205, 1: 11348, 2: 8616, 3: 26315}
[FULL] Train: 66787  Val: 16697
[FULL] Extracting HOG features (train)...




HOG [train]:   0%|          | 0/66787 [00:00<?, ?img/s]

HOG [train]:   0%|          | 64/66787 [00:00<07:15, 153.23img/s]

HOG [train]:   0%|          | 128/66787 [00:00<07:22, 150.73img/s]

HOG [train]:   0%|          | 192/66787 [00:01<07:19, 151.66img/s]

HOG [train]:   0%|          | 256/66787 [00:01<07:11, 154.11img/s]

HOG [train]:   0%|          | 320/66787 [00:02<07:06, 155.76img/s]

HOG [train]:   1%|          | 384/66787 [00:02<07:08, 154.88img/s]

HOG [train]:   1%|          | 448/66787 [00:02<07:14, 152.63img/s]

HOG [train]:   1%|          | 512/66787 [00:03<07:17, 151.41img/s]

HOG [train]:   1%|          | 576/66787 [00:03<07:15, 152.08img/s]

HOG [train]:   1%|          | 640/66787 [00:04<07:19, 150.43img/s]

HOG [train]:   1%|          | 704/66787 [00:04<07:15, 151.73img/s]

HOG [train]:   1%|          | 768/66787 [00:05<07:10, 153.44img/s]

HOG [train]:   1%|          | 832/66787 [00:05<07:12, 152.61img/s]

HOG [train]:   1%|▏         | 896/66787 [00:05<07:13, 152.

[FULL] Train features shape: (66787, 8100)  (took 420.3s)
[FULL] Extracting HOG features (val)...





HOG [val]:   0%|          | 0/16697 [00:00<?, ?img/s]


HOG [val]:   0%|          | 64/16697 [00:00<01:42, 161.76img/s]


HOG [val]:   1%|          | 128/16697 [00:00<01:44, 157.94img/s]


HOG [val]:   1%|          | 192/16697 [00:01<01:44, 158.62img/s]


HOG [val]:   2%|▏         | 256/16697 [00:01<01:42, 160.18img/s]


HOG [val]:   2%|▏         | 320/16697 [00:01<01:41, 162.06img/s]


HOG [val]:   2%|▏         | 384/16697 [00:02<01:40, 161.98img/s]


HOG [val]:   3%|▎         | 448/16697 [00:02<01:39, 163.93img/s]


HOG [val]:   3%|▎         | 512/16697 [00:03<01:38, 164.45img/s]


HOG [val]:   3%|▎         | 576/16697 [00:03<01:39, 162.73img/s]


HOG [val]:   4%|▍         | 640/16697 [00:03<01:39, 161.47img/s]


HOG [val]:   4%|▍         | 704/16697 [00:04<01:39, 160.90img/s]


HOG [val]:   5%|▍         | 768/16697 [00:04<01:38, 161.28img/s]


HOG [val]:   5%|▍         | 832/16697 [00:05<01:37, 162.29img/s]


HOG [val]:   5%|▌         | 896/16697 [00:05<01:37, 161.91img/s]


HOG 

[FULL] Val features shape: (16697, 8100)  (took 104.8s)
[FULL] Will use PCA(n_components=256)
[FULL] Training LinearSVC (C=1.0, dual=False) ...
[FULL] Training time: 34.5s
[FULL] Inference on validation set ...
[FULL] Validation accuracy: 0.7763  (inference 0.5s)
[FULL] TOTAL wall-clock time: 561.1s
[FULL] Saved model to: reports/full_hog_linear_svc.joblib
[FULL] Wrote summary: reports/full_hog_eval.txt
[FULL] Summary: {'train_time_s': 34.452441930770874, 'val_infer_time_s': 0.4702000617980957, 'feat_time_train_s': 420.31419706344604, 'feat_time_val_s': 104.83832287788391, 'total_time_s': 561.0978331565857, 'val_accuracy': 0.7763071210397078, 'model_path': 'reports/full_hog_linear_svc.joblib', 'log_path': 'reports/full_hog_eval.txt', 'train_shape': (66787, 8100), 'val_shape': (16697, 8100)}
